In [ ]:
!pip install transformers[sentencepiece]
!pip install rouge-score nltk
!pip install huggingface_hub
!pip install datasets
!pip install rouge_score
!pip install sacre_bleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sacre_bleu (from versions: none)
ERROR: No matching distribution found for sacre_bleu


In [ ]:
import pandas as pd
import numpy as np

import transformers
from transformers import pipeline, set_seed

import tensorflow as tf 
import datasets

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from datasets import load_metric
metric = load_metric("rouge")

<ipython-input-3-ffe5587f2e49>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
df = pd.read_csv('/content/FF_dataset.csv')
df

,notes,email summary,country
0,"36-year-old engineer, been working there for 5...","Dear [Name], \n\nThank you for providing me wi...",UK
1,I often tell clients that planning for their r...,"Dear [Client], \n\nThank you for your recent c...",UK
2,"Name - Scott Snedden Age 45 Employed £100,000","Dear Scott, \n\nI hope this email finds you we...",UK
3,"Family D.O.B: 47 Married / Single: Kate, 40 D....","Dear Name, \n\nI wanted to summarize the fact-...",UK
4,"Name: Joe Young 50 years old Single, no depend...","Dear [username], \n\nI am writing to summarise...",UK
5,Note from meeting Helen Sims.Referred by Sharo...,Subject: Financial Fact-Find Summary for Helen...,UK
6,Dr Alistair Forbes and wife Jackie. Referred b...,Subject: Fact-find Summary for Dr. Alistair Fo...,UK
7,Kids eduction Tax planning Mega back door roth...,"Dear [username], \n\nI have reviewed the infor...",US
8,Name: Kyle Smith Age: 38 Occupation: Financial...,"Dear Kyle, \n\nI have reviewed the financial i...",US
9,Client Marital status:Married Occupation: Curr...,"Subject: Summary of Fact-Find \n\nDear Client,...",US


In [ ]:
df.drop(index=1, inplace=True)
df.reset_index(inplace=True, drop=True)

In [ ]:
import re
#extract the main summary i.e. "positive/negative areas" from the mail and drop the rest of the text
#this is done so that we can get an accurate ROUGE metric when comparing model and actual summary texts.
t = df['email summary'][0]
print('Email:', t[:100])
t = re.findall("(?<=areas:)[\s\S]*(?=:)|(?<=Areas:)[\s\S]*(?=:)|(?<=Aspects:)[\s\S]*(?=:)", t)
print('Client Summary:', t[0][:100])
t = "\n".join(t[0].split('\n')[:-1])
t = re.sub('\n.*:','',t)
print('Client Summary Final:', t[:100])

Email: Dear [Name], 

Thank you for providing me with the details of your financial situation. I appreciate
Client Summary:  

You are a 36-year-old engineer who has been working for 5 years. 

You have a non-qualified pensi
Client Summary Final:  

You are a 36-year-old engineer who has been working for 5 years. 

You have a non-qualified pensi


In [ ]:
def get_summary(text):
    text = re.findall("(?<=areas:)[\s\S]*(?=:)|(?<=Areas:)[\s\S]*(?=:)|(?<=Aspects:)[\s\S]*(?=:)", text)
    text = text[0]
    try:
        text = "\n".join(text.split('\n')[:-1])
    except:
        pass
    text = re.sub('\n.*:','',text)
    return text

df['summary'] = df['email summary'].apply(lambda x: get_summary(x))

## **Transformer Based models pre-tuned on text summarization task:**
### **1. T5**
### **2. BART**
### **3. PEGASUS**

Since the Dataset is very small we cannot use it for fine-tuning any of these models. 

One of the best performing models is ChatGPT built over GPT-3 but we cannot use it because it's api isn't for free use.

### **1. T5 Implementation**

In [ ]:
t5_summarizer = pipeline('summarization', model = 't5-small')
t = df['notes'][0]
mod_sum = t5_summarizer(t, min_length=len(t.split(' '))//2, max_length=len(t.split(' ')))

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon

In [ ]:
"\n".join(sent_tokenize(mod_sum[0]['summary_text']))

'engineer, 36, is about to move back to the uk for family reasons .\nowns a property about to sell .\na non-qualified pension with €120k in there .\nstill french resident, in the UK on 6th April 2023 for tax provisionally .'

In [ ]:
def summarize_text(summarizer, text):
    textlen = len(text.split(' '))
    try:
        mod_sum = summarizer(text, min_length=textlen//2, max_length=textlen)
    except:
        mod_sum = summarizer(text, min_length=textlen//2, max_length=512)
    return "\n".join(sent_tokenize(mod_sum[0]['summary_text']))

In [ ]:
df['t5_small'] = df['notes'].apply(lambda x: summarize_text(t5_summarizer, x))

Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

results_t5 = metric.compute(predictions = df['t5_small'], references = df['summary'])
results_t5_ = dict((rn, results_t5[rn].mid.fmeasure) for rn in rouge_names)
results_t5_

{'rouge1': 0.22964928458088243,
 'rouge2': 0.09806754285899952,
 'rougeL': 0.15825040922491737,
 'rougeLsum': 0.21109690156880362}

### **2. BART Implementation**

In [ ]:
import torch

In [ ]:
torch.cuda.current_device()

0

In [ ]:
bart_summarizer = pipeline('summarization', model = 'facebook/bart-large-cnn', device="cuda:0")
t = df['notes'][0]
mod_sum = bart_summarizer(t, min_length=len(t.split(' '))//2, max_length=len(t.split(' ')))

In [ ]:
"\n".join(sent_tokenize(mod_sum[0]['summary_text']))

'36-year-old engineer, been working there for 5 years, about to move back to UK for family reasons.\nOwn a property about to sell.\nNon-qualified pension with EFA organisation, €120k in there (more of an investment fund rather than a pension), already an agreement with French tax authorities.'

In [ ]:
df['bart_large_cnn'] = df['notes'].apply(lambda x: summarize_text(bart_summarizer, x))

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

results_bart = metric.compute(predictions = df['bart_large_cnn'], references = df['summary'])
results_bart_ = dict((rn, results_bart[rn].mid.fmeasure) for rn in rouge_names)
results_bart_

{'rouge1': 0.22951866996467207,
 'rouge2': 0.09658183970416549,
 'rougeL': 0.15951114053296567,
 'rougeLsum': 0.21520074513605408}

### **3. PEGASUS Implementation**

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sentencepiece

In [ ]:
peg_summarizer = pipeline('summarization', model = 'google/pegasus-xsum', device="cuda:0")
t = df['notes'][0]
mod_sum = peg_summarizer(t, min_length=len(t.split(' '))//2, max_length=len(t.split(' ')))

In [ ]:
"\n".join(sent_tokenize(mod_sum[0]['summary_text']))

'A French resident working in the UK on a non-qualified pension with EFA organisation, 120k in there (more of an investment fund rather than a pension), already an agreement with French tax authorities who won’t tax the income from salary but will tax the capital gains (but quite small).'

In [ ]:
df['pegasus_finsum'] = df['notes'].apply(lambda x: summarize_text(peg_summarizer, x))

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

results_peg = metric.compute(predictions = df['pegasus_finsum'], references = df['summary'])
results_peg_ = dict((rn, results_bart[rn].mid.fmeasure) for rn in rouge_names)
results_peg_

{'rouge1': 0.1344050121501129,
 'rouge2': 0.03914136069286434,
 'rougeL': 0.09902390956512333,
 'rougeLsum': 0.1190789857021321}

## **T5 Large Implementation**

In [ ]:
t5_summarizer = pipeline('summarization', model = 't5-large', device='cuda:0')
t = df['notes'][0]
mod_sum = t5_summarizer(t, min_length=len(t.split(' '))//2, max_length=len(t.split(' ')))

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon

In [ ]:
"\n".join(sent_tokenize(mod_sum[0]['summary_text']))

'36-year-old engineer about to move back to the uk for family reasons .\nalready an agreement with french tax authorities who won’t tax the income from salary but will tax the capital gains .\nstill french resident, in the UK on 6th April 2023 for tax provisionally and physically moving back around 15th February .'

In [ ]:
def summarize_text(summarizer, text):
    textlen = len(text.split(' '))
    try:
        mod_sum = summarizer(text, min_length=textlen//2, max_length=textlen)
    except:
        mod_sum = summarizer(text, min_length=textlen//2, max_length=512)
    return "\n".join(sent_tokenize(mod_sum[0]['summary_text']))

In [ ]:
df['t5_large'] = df['notes'].apply(lambda x: summarize_text(t5_summarizer, x))

Token indices sequence length is longer than the specified maximum sequence length for this model (581 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

results_t5 = metric.compute(predictions = df['t5_large'], references = df['summary'])
results_t5_ = dict((rn, results_t5[rn].mid.fmeasure) for rn in rouge_names)
results_t5_

{'rouge1': 0.2357096806228829,
 'rouge2': 0.10834806066068364,
 'rougeL': 0.16594264729889815,
 'rougeLsum': 0.2169242670896868}

## **Summary**

In [ ]:
# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Model", "score", "metric", "rouge1", "rouge2", "rougeL", "rougeLsum"])
  
# Add rows
myTable.add_row(["T5-Small", "mid", "fmeasure", "0.23", "0.09", "0.16", "0.21"])
myTable.add_row(["BART", "mid", "fmeasure", "0.23", "0.09", "0.16", "0.21"])
myTable.add_row(["PEGASUS", "mid", "fmeasure", "0.13", "0.03", "0.09", "0.11"])
myTable.add_row(["T5-Large", "mid", "fmeasure", "0.24", "0.10", "0.17", "0.22"])

  
print(myTable)

+----------+-------+----------+--------+--------+--------+-----------+
|  Model   | score |  metric  | rouge1 | rouge2 | rougeL | rougeLsum |
+----------+-------+----------+--------+--------+--------+-----------+
| T5-Small |  mid  | fmeasure |  0.23  |  0.09  |  0.16  |    0.21   |
|   BART   |  mid  | fmeasure |  0.23  |  0.09  |  0.16  |    0.21   |
| PEGASUS  |  mid  | fmeasure |  0.13  |  0.03  |  0.09  |    0.11   |
| T5-Large |  mid  | fmeasure |  0.24  |  0.10  |  0.17  |    0.22   |
+----------+-------+----------+--------+--------+--------+-----------+
